In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def generate_text(prompt):
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids)
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=False)
    return generated_text

prompt = "I want to order a cup of coffee and"
result = generate_text(prompt)
print(result)

/Users/hyuk/anaconda3/envs/pytorchLecture/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/Users/hyuk/anaconda3/envs/pytorchLecture/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


I want to order a cup of coffee and I want to order a cup of tea. I want


In [2]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel

torch.manual_seed(42)

def generate_text(prompt):
    # Initialize model and tokenizer
    model_name = 'gpt2'
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.eval()

    # Generate text
    generated_text = prompt
    for i in range(50): # Generate 100 tokens
        with torch.no_grad():
            input_ids = tokenizer.encode(generated_text, return_tensors='pt')
            outputs = model(input_ids)
            next_token_logits = outputs.logits[:, -1, :]
            
            next_token_probs = F.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(next_token_probs, num_samples=1)
           
            input_ids = torch.cat([input_ids, next_token], dim=1)
            
            next_word = tokenizer.decode(next_token[0])
            generated_text += next_word
            print(generated_text)


    return generated_text

prompt = "I want to order a cup of coffee and"
result = generate_text(prompt)

I want to order a cup of coffee and I
I want to order a cup of coffee and I'm
I want to order a cup of coffee and I'm f
I want to order a cup of coffee and I'm fiddling
I want to order a cup of coffee and I'm fiddling around
I want to order a cup of coffee and I'm fiddling around here
I want to order a cup of coffee and I'm fiddling around here in
I want to order a cup of coffee and I'm fiddling around here in my
I want to order a cup of coffee and I'm fiddling around here in my chair
I want to order a cup of coffee and I'm fiddling around here in my chair trying
I want to order a cup of coffee and I'm fiddling around here in my chair trying to
I want to order a cup of coffee and I'm fiddling around here in my chair trying to pass
I want to order a cup of coffee and I'm fiddling around here in my chair trying to pass the
I want to order a cup of coffee and I'm fiddling around here in my chair trying to pass the time
I want to order a cup of coffee and I'm fiddling around here in my cha

In [3]:
import torch
import torch.nn.functional as F
from transformers import GPT2LMHeadModel
import tiktoken

torch.manual_seed(42)

def generate_text(prompt):
    # Initialize model and tokenizer
    model_name = 'gpt2'
    tokenizer = tiktoken.encoding_for_model(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.eval()

    # Generate text
    generated_text = prompt
    for i in range(50):  # Generate 50 tokens
        with torch.no_grad():
            input_ids = torch.tensor([tokenizer.encode(generated_text)])
            outputs = model(input_ids)
            next_token_logits = outputs.logits[:, -1, :]
            
            next_token_probs = F.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(next_token_probs, num_samples=1)
           
            next_word = tokenizer.decode([next_token.item()])
            generated_text += next_word
            # print(generated_text)

    return generated_text

prompt = "I want to order a cup of coffee and"
result = generate_text(prompt)
print(f"\nFinal generated text:\n{result}")


Final generated text:
I want to order a cup of coffee and I'm fiddling around here in my chair trying to pass the time like a normal person...to finish dessert for dinner)."


Lurky Girl goes on several episodes of REAL CRIMINAL SEXITY as she has an hissy


In [4]:
def generate_text(prompt, model):
    model_name = 'gpt2'
    tokenizer = tiktoken.encoding_for_model(model_name)
    model.to(torch.device('cpu'))
    model.eval()

    generated_text = prompt
    for i in range(50):  # Generate 50 tokens
        with torch.no_grad():
            input_ids = torch.tensor([tokenizer.encode(generated_text,disallowed_special=())])
            outputs = model(input_ids)
            next_token_logits = outputs.logits[:, -1, :]
            
            next_token_probs = F.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(next_token_probs, num_samples=1)
           
            next_word = tokenizer.decode([next_token.item()])
            generated_text += next_word

    return generated_text

In [5]:

print(torch.__version__)
if torch.backends.mps.is_available():
    my_device = torch.device('mps')
elif torch.cuda.is_available():
    my_device = torch.device('cuda')
else:
    my_device = torch.device('cpu')
print(my_device)


2.2.0
mps


In [6]:
with open("bible.txt", 'r', encoding='utf-8') as f:
    text = f.read()
    tokenizer = tiktoken.encoding_for_model('gpt2')
    tokens = tokenizer.encode(text)
bible_tokens = input_ids = torch.tensor(tokens, dtype=torch.long)
print(bible_tokens.shape)

torch.Size([1258315])


In [7]:

def get_batch(data, batch_size, block_length):
    # generate a small batch of data of inputs x and targets y
    
    ix = torch.randint(len(data) - block_length, (batch_size,))
    batch_input = torch.stack([data[i:i+block_length] for i in ix])
    batch_target = torch.stack([data[i+1:i+block_length+1] for i in ix])
    return batch_input, batch_target

batch_input, batch_target = get_batch(bible_tokens, batch_size=1, block_length=1024) #try batch_size=4
print(batch_input)
print(batch_target)

tensor([[15350,   656,   262,  ...,  2607,    25,   198]])
tensor([[  656,   262,  2769,  ...,    25,   198, 22834]])


In [8]:
import torch.nn as nn
from torch.optim import Adam
torch.manual_seed(42)
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(my_device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-5)


for steps in range(100000):
    input_batch, target_batch = get_batch(bible_tokens, batch_size=4, block_length=1024)
    input_batch = input_batch.to(my_device)
    target_batch = target_batch.to(my_device)

    logits = model(input_batch).logits

    logits = logits.view(-1, logits.size(-1))
    target_batch = target_batch.view(-1)
    
    loss = criterion(logits,target_batch)
   
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if steps % 10 == 0:
        print("Step: {}, Loss: {}".format(steps, loss.item()))
        prompt = "In the beginning,"
        print(generate_text(prompt, model))
        model.to(my_device)
        model.train()   

Step: 0, Loss: 8.116193771362305
In the beginning, states created the US Unified Features — requirements that defined less oppressive political systems. Now it's all about the feds controlling the first 2 percent of the population. These people can't breathe, start voting, limit political freedom at a young age, and think
Step: 10, Loss: 6.831602573394775
In the beginning, there was lots of allegations I never thought I would get and a lot of anxiety.


Will Oni-Ra


Crossref Citations


I was fortunate enough to have some religious friends but not much of either religion
Step: 20, Loss: 6.167196750640869
In the beginning, the problem that surrounded the application of a Stewart-Marneault report as a 1865 report is this: it lacked cogent evidence. He detailed how Stewart brought rip-roaring sail operators with their plant excavation tools. He also followed up his own
Step: 30, Loss: 5.926900386810303
In the beginning, they all worked, on rocks come and go. The stars rose higher. Blue. 